# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# load csv output_data_file from WeatherPy exercise
data_df = pd.read_csv(r'C:\Users\Lenovo\Desktop\python-api-challenge\WeatherPy\output_data\cities.csv')
data_df = data_df.dropna()
data_df.head()


,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,tshela,100.0,CD,1.583907e+09,97.0,-4.98,12.93,74.61,1.25
1,lufilufi,97.0,WS,1.583907e+09,70.0,-13.87,-171.60,87.01,4.00
2,saskylakh,0.0,RU,1.583907e+09,83.0,71.92,114.08,-15.48,17.76
3,tuatapere,61.0,NZ,1.583907e+09,69.0,-46.13,167.68,55.99,3.00
4,butaritari,18.0,KI,1.583907e+09,77.0,3.07,172.79,84.00,20.74


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = data_df[["Lat", "Lng"]].astype(float)

humidity = data_df["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
ideal_data_df = data_df[data_df["Max Temp"] > 70]
ideal_data_df = ideal_data_df[ideal_data_df["Max Temp"] < 80]
ideal_data_df = ideal_data_df[ideal_data_df["Wind Speed"] < 10]
ideal_data_df = ideal_data_df[ideal_data_df["Cloudiness"] == 0]

ideal_data_df = ideal_data_df.dropna()

ideal_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
42,san patricio,0.0,MX,1.583907e+09,77.0,19.22,-104.70,75.88,2.89
131,tecoanapa,0.0,MX,1.583907e+09,83.0,16.52,-98.75,73.42,3.42
186,pozo colorado,0.0,PY,1.583907e+09,41.0,-23.49,-58.80,77.56,6.89
203,sao filipe,0.0,CV,1.583907e+09,77.0,14.90,-24.50,70.68,4.97
334,maua,0.0,BR,1.583907e+09,93.0,-23.67,-46.46,77.00,1.12
341,santa rosa de viterbo,0.0,BR,1.583907e+09,83.0,-21.47,-47.36,71.60,4.70
390,abu dhabi,0.0,AE,1.583907e+09,53.0,24.47,54.37,73.00,6.93
399,huarmey,0.0,PE,1.583907e+09,71.0,-10.07,-78.15,71.53,1.21
449,inhambane,0.0,MZ,1.583907e+09,70.0,-23.86,35.38,79.27,4.41
456,coahuayana,0.0,MX,1.583907e+09,67.0,18.73,-103.68,72.46,1.23


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map